In [190]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [210]:
def calculate_change(price, big_bill=1000, denoms=[500, 200, 100, 50, 20, 10, 5]):
    """
    Returns a dictionary of how many of each coin/bill will be needed for a given price and bill used to pay.
    
    Assumptions:
    - The customer pays with the smallest number of 'big_bill' notes required to cover the price, and no small coins.
    - The cashier uses the greedy algorithm (always giving the largest denomination possible).
    - Works optimally for most real-world currencies (AUD, USD, GBP, EUR).
    - All amounts are in cents.
    """
    # how much the customer pays (in cents)
    amount_paid = ((price + big_bill - 1) // big_bill) * big_bill  # round up to next big_bill
    change_amount = amount_paid - price

    # initialise dictionary of change counts
    change_paid = dict.fromkeys(denoms, 0)

    remaining = change_amount
    for coin in denoms:
        if remaining == 0:
            break
        count = remaining // coin        # number of this coin
        change_paid[coin] = count
        remaining -= count * coin        # reduce the remaining change

    return [change_paid[i] for i in denoms] # as a list for simplicity

def generate_change_matrix(items, big_bill=1000, denoms=[500, 200, 100, 50, 20, 10, 5]):
    """
    creates a matrix representing how much change is used for each price
    col index = an item's index, row index = the denomination's index
    """
    prices = items["Price"]
    # a matrix which tells us at Fij the amount of change of denom i for a sale of a price j
    uses_per_price = np.array([calculate_change(i, big_bill, denoms) for i in prices]).T

    # product of the matrices to calculate the change usage
    return uses_per_price

def generate_sales_figs(items, n=10**6):
    """
    generates sales figures as an array
    each col is one day's sales of each item
    """
    means = items["Mean_Sales"]
    samples=[]
    for mean in means:
        samples.append(np.random.poisson(lam=mean, size=n)) # generate random amounts of sales
    samples = np.array(samples)
    return samples

def monte_carlo_change(items, n=10**6, big_bill=1000, denoms=[500,200,100,50,20,10,5]):
    """
    uses generated sales figures to generate data about the change used each day
    """
    figs = generate_sales_figs(items, n)
    change_matrix = generate_change_matrix(items, big_bill, denoms)
    # use dot product to let library do the heavy lifting (using python loops this could take minutes to hours)
    return np.dot(change_matrix,figs)

def count_fails(results, float_amount):
    """
    given an array of daily results, check how many times we ran out of cash, 
    return result as percentage
    """
    fails = 0
    # for each day
    for i in results.T:
        # calculate remaining cash of each denomination
        remainder = float_amount-i
        
        # see if any values are negative ie. we ran out of any cash today
        for j in remainder:
            if j<0:
                fails+=1
                break
    return str(100.0*fails/len(results.T))+"%"

def probability_of_failure(items, float_amount, n=10**6, big_bill=1000, denoms=[500,200,100,50,20,10,5]):
    """
    given some items on the menu, calculate the probability that we run out of cash 
    given the cash float
    """
    results = monte_carlo_change(items, n, big_bill, denoms)
    return count_fails(results, float_amount)

def most_used_denom(results, denoms):
    means = []
    for i in results:
        means.append(np.mean(i))
    return denoms[means.index(max(means))]

In [213]:
# items on the menu, their prices, and mean number of sales
items = pd.DataFrame({
    "Price":[720,1330,2710,800,1550,2870],
    "Mean_Sales":[50,60,20,35,45,12],
})
# denominations in cents (AUD used here)
denoms = [500,200,100,50,20,10,5]
# amount of cash we start the day with, in each denomination
float_amount = np.array([200,250,300,300,300,300,300])
# what bill are the customers going to pay with (assume customers use only big bills)
big_bill=1000


# desired number of times to run the monte carlo simulation
n=10**6

In [ ]:
# let's estimate the probability of running out of a denomination during a day's operations:
results = monte_carlo_change(items=items,n=n,big_bill=big_bill,denoms=denoms)
# find out what the most used coin/note was
most_used = most_used_denom(results, denoms)
failure_prob = count_fails(results, float_amount)

print(f"The most used denomination was {most_used}")
print(f"The probability of running out of cash is {failure_prob}")



The most used denomination was 200
The probability of running out of cash is 0.0813%
